In [1]:
import pandas as pd
from functools import reduce
import matplotlib.pyplot as plt

In [2]:
file_path = '../data/nba_points_2024_2025.xlsx'
df = pd.read_excel(file_path)

In [3]:
print('GAMES TRACKED: ', round(df['Game Count'].sum()))
print('AVG. OPEN TOTAL: ', round(df['Open Total'].mean(), 2))
print('AVG. ACTUAL TOTAL: ', round(df['Actual Total'].mean(), 2))
print('OVERS: ', round(df['Over'].sum()/df['Game Count'].sum() * 100, 2), '%')
print('UNDERS: ', round(df['Under'].sum()/df['Game Count'].sum() * 100, 2), '%')

GAMES TRACKED:  515
AVG. OPEN TOTAL:  225.72
AVG. ACTUAL TOTAL:  225.99
OVERS:  51.07 %
UNDERS:  48.74 %


In [4]:
game_counts = df.groupby(['Crew Chief'])['Game Count'].sum().reset_index(name='games')
over_percs = round(df.groupby(['Crew Chief'])['Over'].sum()/df.groupby('Crew Chief')['Game Count'].sum() * 100, 2).reset_index(name='over_percentage')
over_by_avg = round(df.groupby(['Crew Chief'])['Over Amount'].mean(), 2).reset_index(name='over_by_avg')
over_by_std = round(df.groupby(['Crew Chief'])['Over Amount'].std(), 2).reset_index(name='over_by_std')
under_by_avg = round(df.groupby(['Crew Chief'])['Under Amount'].mean(), 2).reset_index(name='under_by_avg')
under_by_std = round(df.groupby(['Crew Chief'])['Under Amount'].std(), 2).reset_index(name='under_by_std')
under_percs = round(df.groupby(['Crew Chief'])['Under'].sum()/df.groupby('Crew Chief')['Game Count'].sum() * 100, 2).reset_index(name='under_percentage')
avg_totals = round(df.groupby(['Crew Chief'])['Actual Total'].mean(), 2).reset_index(name='mean_actual_total')
std_totals = round(df.groupby(['Crew Chief'])['Actual Total'].std(), 2).reset_index(name='std_actual_total')
avg_opening_total = round(df.groupby(['Crew Chief'])['Open Total'].mean(), 2).reset_index(name='mean_open_total')

groupby_results = [game_counts, over_percs, over_by_avg, over_by_std, under_by_avg, under_by_std, under_percs, avg_totals, std_totals, avg_opening_total]
chief_stats = reduce(lambda left, right: pd.merge(left, right, on='Crew Chief'), groupby_results)

In [5]:
chief_stats.columns

Index(['Crew Chief', 'games', 'over_percentage', 'over_by_avg', 'over_by_std',
       'under_by_avg', 'under_by_std', 'under_percentage', 'mean_actual_total',
       'std_actual_total', 'mean_open_total'],
      dtype='object')

In [7]:
chief_stats[(chief_stats.games >= 15) &
            ((chief_stats.over_percentage > 66) | (chief_stats.under_percentage > 66))  ][['Crew Chief', 'games', 'over_percentage', 'under_percentage']].sort_values(by='over_percentage')

,Crew Chief,games,over_percentage,under_percentage
7,Ed Malloy,26.0,23.08,76.92
22,Pat Fraher,17.0,29.41,70.59
20,Mitchell Ervin,19.0,68.42,31.58
24,Scott Foster,21.0,76.19,23.81
